In [149]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import regex as re

In [150]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://open.overheid.nl/zoekresultaten?datumBeschikbaarVanaf=01-01-2019&text=woo%20verzoek&informatiesoort=c_4f50ca9c&page=1&count=100")

title = driver.title

Dataframe = None

column=['Titel', 'Verzoek', 'Specifiek', 'Verantwoordelijke', 'Thema', 'Documentsoort', 'Publicatiedatum', 'Documentdatum', 'Onderwerp']

lists = driver.find_elements(By.CLASS_NAME, 'result--title ')

URLS = []
for i in lists:
    element = i.find_element(By.TAG_NAME, 'a').get_attribute("href")
    URLS.append(element)

for i in URLS:
    driver.get(i)
    title = (driver.find_element(By.CLASS_NAME, "h2")).text
    split = title.split(" - ")

    if len(split) == 1:
        Row = [title, split[0], np.nan]
    else:
        Row = [title, split[0], split[1]]
    
    table = driver.find_element(By.CLASS_NAME, "table__data-overview")
    
    for row in table.find_elements(By.XPATH, ".//tr"):
        row_data = row.find_element(By.XPATH, ".//td").get_attribute('textContent')
        
        if ('\n') in row_data:
            Data = []
            
            i = row_data.split('\n')
            for item in i:
                res = re.findall(r'\w+', item)
                if res != []:
                    sentence = ' '.join([str(r) for r in res])
                    Data.append(sentence)
        else:
            Data = row_data
        
        Row.append(Data)
        

    if Dataframe is None:
        Dataframe = pd.DataFrame(columns=column)

    if len(Row) == 8:
        Row.append(np.nan)
        
    Dataframe.loc[len(Dataframe)] = Row
    

driver.quit()

display(Dataframe)

,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp
0,Besluit op Woo-verzoek inzake het wokisme waar...,Besluit op Woo-verzoek inzake het wokisme waar...,Inventarisatielijst Woo-verzoek wokisme,Ministerie van Justitie en Veiligheid,Onbekend thema,Beslissing op Wob-/Woo-verzoek,21-02-2023,21-02-2023,NaN
1,Besluit op Woo-verzoek inzake het wokisme waar...,Besluit op Woo-verzoek inzake het wokisme waar...,Besluit op Woo-verzoek inzake het wokisme,Ministerie van Justitie en Veiligheid,Onbekend thema,Beslissing op Wob-/Woo-verzoek,21-02-2023,21-02-2023,NaN
2,Besluit op Woo-verzoek om informatie inzake he...,Besluit op Woo-verzoek om informatie inzake he...,Documenten bij Woo-verzoek,Ministerie van Justitie en Veiligheid,Onbekend thema,Beslissing op Wob-/Woo-verzoek,20-02-2023,20-02-2023,NaN
3,Besluit op Woo-verzoek om informatie inzake he...,Besluit op Woo-verzoek om informatie inzake he...,Besluit op Woo-verzoek om informatie inzake he...,Ministerie van Justitie en Veiligheid,Onbekend thema,Beslissing op Wob-/Woo-verzoek,20-02-2023,20-02-2023,NaN
4,Besluit op Woo-verzoek inzake het wokisme waar...,Besluit op Woo-verzoek inzake het wokisme waar...,NaN,Ministerie van Justitie en Veiligheid,Onbekend thema,Beslissing op Wob-/Woo-verzoek,21-02-2023,21-02-2023,NaN
...,...,...,...,...,...,...,...,...,...
95,Woo-verzoek over ongeval met trampoline,Woo-verzoek over ongeval met trampoline,NaN,"Ministerie van Landbouw, Natuur en Voedselkwal...",Onbekend thema,Beslissing op Wob-/Woo-verzoek,23-08-2022,23-08-2022,NaN
96,Aanvullend besluit Woo-verzoek overgangsrecht ...,Aanvullend besluit Woo-verzoek overgangsrecht ...,Aanvullend besluit Woo-verzoek overgangsrecht ...,Ministerie van Financiën,Belasting,Beslissing op Wob-/Woo-verzoek,13-12-2022,07-12-2022,Belasting betalen
97,Besluit op Woo-verzoek over omroep Ongehoord N...,Besluit op Woo-verzoek over omroep Ongehoord N...,NaN,"Ministerie van Onderwijs, Cultuur en Wetenschap",Media,Beslissing op Wob-/Woo-verzoek,31-01-2023,31-01-2023,Media en publieke omroep
98,2e deelbesluit op Wob-Woo-verzoek over Tijdeli...,2e deelbesluit op Wob-Woo-verzoek over Tijdeli...,2e deelbesluit op Wob-Woo-verzoek over Tijdeli...,Ministerie van Binnenlandse Zaken en Koninkrij...,"[Openbare orde en veiligheid, Veiligheid]",Beslissing op Wob-/Woo-verzoek,15-02-2023,13-02-2023,[Bevoegdheden inlichtingendiensten en veilighe...
